In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

from modules.info import *
from modules.save2csv import *


In [2]:
df1 = pd.read_csv(path + 'outputs/0_处理好的主体_' + date + '.csv', index_col=[0, 1])
df2 = pd.read_excel(path + 'rules/指标字典_230526.xlsx')
df2 = df2.sort_values('优先级')
df2 = df2[['代码', '优先级', '代码公式']]
df2.dropna(inplace=True)


In [3]:
df2.rename(columns={'代码': 'code',
                    '优先级': 'p',
                    '代码公式': 'code_cal'}, inplace=True)
df2['p'] = pd.to_numeric(df2['p'])
df2_1 = df2[df2['p'] == 1]
df2_2 = df2[df2['p'] == 2]
df2_3 = df2[df2['p'] == 3]

In [4]:
rule_dict1 = {}
for index, row in df2_1.iterrows():
    code = row['code']
    formula = row['code_cal']
    # 使用正则表达式提取指标
    # 匹配：所有正常指标
    pattern = r"([a-z]{3,9}+\_[A-Z]{1}+[a-z]+\^?[1-3]?_?[0-9]{0,3})|([a-zA-Z]+\_[0-9]{1,5}\^?[1-3]?\_?[0-9]{1,2}\^?[1-3]?)|(EBITDA)|(EBIT)|(EN_ANNUAL_MarketCapitalization_idou)"
    result1 = re.sub(pattern, lambda x: f'df1[\'{x.group()}\']', formula)
    print(result1)
    rule_dict1[code] = result1

print('===== 开始计算p1指标 =====')

results1 = {}
for code, formula in rule_dict1.items():

    # 修正提前期标记
    for i in range(1, 4):
        formula = formula.replace(f"^{i}']", f"'].shift({i})")

    # 尝试指标计算，打印会导致报错的指标
    try:
        result1 = eval(formula)
    
    except Exception as e:
        print(code, ': {}'.format(type(e).__name__))
    
    # 汇总输出计算结果字典results
    results1[code] = pd.DataFrame(result1).reset_index()

for code, result1 in results1.items():
    result1.rename(columns={result1.columns[2]: code}, inplace=True)

for code, result1 in results1.items():
    df1 = pd.merge(df1, result1, on=['entity_id', 'year'], how='left')

print('===== p1指标计算完成 =====')


(df1['PF_0102']+df1['PF_0102^1'])/2
df1['EN_ANNUAL_MarketCapitalization_idou']
df1['BS_0499']/df1['BS_0299']
df1['stmnote_Rdexp']+df1['stmnote_Rdexp^1']
(df1['stmnote_Rdexp']+df1['stmnote_Rdexp^1'])/(df1['PF_0102']+df1['PF_0102^1'])
df1['PF_0129']+df1['PF_0118']-df1['PF_0118_1']
df1['PF_0125']/df1['PF_0101']
df1['CF_0112']/df1['PF_0132']
df1['PF_0129']/(df1['PF_0107']+df1['PF_0115']+df1['PF_0116']+df1['PF_0117']+df1['PF_0118']+df1['PF_0139'])
2*df1['PF_0134']/((df1['BS_0501']+df1['BS_0502'])+(df1['BS_0501^1']+df1['BS_0502^1']))
2*df1['PF_0101']/(df1['BS_0299']+df1['BS_0299^1'])
2*df1['PF_0101']/(df1['BS_0104']+df1['BS_0104^1'])
2*df1['CF_0112']/(df1['BS_0299']+df1['BS_0299^1'])
df1['CF_0112']/df1['BS_0391']
(df1['BS_0301']+df1['BS_0303']+df1['BS_0315']+df1['BS_0317_8']+df1['BS_0401']+df1['BS_0402']+df1['BS_0312'])/df1['BS_0499']
(df1['PF_0125']-df1['PF_0125^1'])/abs(df1['PF_0125^1'])
(df1['BS_0299']-df1['BS_0299^1'])/abs(df1['BS_0299^1'])
2*df1['PF_0107']/(df1['BS_0109']+df1['BS_0109^1

In [5]:
rule_dict2 = {}
for index, row in df2_2.iterrows():
    code = row['code']
    formula = row['code_cal']
    # 使用正则表达式提取指标
    pattern = r"([a-zA-Z]{4,30}\^?[1-3]?)|([a-zA-Z]+\_[0-9]{1,5}\^?[1-3]?\_?[0-9]{1,2}\^?[1-3]?)|(ROE\^?[1-3]?)|(Sum3YRDInvestment)|(Sum3YRevenue)"
    result2 = re.sub(pattern, lambda x: f'df1[\'{x.group()}\']', formula)
    print(result2)
    rule_dict2[code] = result2

df1.reset_index()
df1.set_index([df1.columns[0], df1.columns[1]], inplace=True)

print('===== 开始计算p2指标 =====')

results2 = {}
for code, formula in rule_dict2.items():

    # 修正提前期标记
    for i in range(1, 3):
        formula = formula.replace(f"^{i}']", f"'].shift({i})")

    # 尝试指标计算，打印会导致报错的指标
    try:
        result2 = eval(formula)
        
    except Exception as e:
        print(code, ': {}'.format(type(e).__name__))

    # 汇总输出计算结果字典results
    results2[code] = pd.DataFrame(result2).reset_index()

for code, result2 in results2.items():
    result2.rename(columns={result2.columns[2]: code}, inplace=True)

for code, result2 in results2.items():
    df1 = pd.merge(df1, result2, on=['entity_id', 'year'], how='left')

# 特殊情况处理
# MinProfitAndProfitAfterDedu = min(df1['PF_0132'], df1['TempProfitAfterDeduction'])
try:
    df1['MinProfitAndProfitAfterDedu'] = df1[['PF_0132','TempProfitAfterDeduction']].min(axis=1)
    print('MinProfitAndProfitAfterDedu : 已修正')
except:
    print('= ERROR =')

print('===== p2指标计算完成 =====')


(df1['BS_0299']-df1['IntBearLiability']-df1['BS_0501']-df1['BS_0502'])/(df1['BS_0299']-df1['BS_0307']-df1['BS_0308'])
df1['EBIT']/df1['PF_0118_1']
(df1['ROE']+df1['ROE^1'])/2
(df1['PF_0116']+df1['PF_0117']+df1['PF_0118']+df1['PF_0139'])/(df1['PF_0102']-df1['PF_0102_11'])
df1['EBITDA']/df1['PF_0101']
(df1['PF_0102']-df1['PF_0107'])/(df1['PF_0102']-df1['PF_0102_11'])
(df1['PF_0107']+df1['PF_0115']+df1['PF_0116']+df1['PF_0117']+df1['PF_0118']+df1['PF_0139'])/(df1['PF_0102']-df1['PF_0102_11'])
(df1['IntBearLiability']-df1['IntBearLiability^1'])/abs(df1['IntBearLiability^1'])
df1['IntBearLiability']/df1['BS_0591']
df1['DistributableProfits']/df1['PF_0118']-df1['PF_0118_1']
2*df1['EBIT']/(df1['BS_0299']+df1['BS_0299^1'])
min(df1['PF_0132'],df1['TempProfitAfterDeduction'])
df1['Sum3YRDInvestment']/df1['Sum3YRevenue']
===== 开始计算p2指标 =====
MinProfitAndProfitAfterDedu : ValueError
MinProfitAndProfitAfterDedu : 已修正
===== p2指标计算完成 =====


In [6]:
rule_dict3 = {}
for index, row in df2_3.iterrows():
    code = row['code']
    formula = row['code_cal']
    # 使用正则表达式提取指标
    pattern = r"([a-zA-Z]{4,35}\^?[1-3]?)"
    result3 = re.sub(pattern, lambda x: f'df1[\'{x.group()}\']', formula)
    print(result3)
    rule_dict3[code] = result3

df1.reset_index()
df1.set_index([df1.columns[0], df1.columns[1]], inplace=True)

print('===== 开始计算p3指标 =====')

results3 = {}
for code, formula in rule_dict3.items():

    # 修正提前期标记
    for i in range(1, 3):
        formula = formula.replace(f"^{i}']", f"'].shift({i})")

    # 尝试指标计算，打印会导致报错的指标
    try:
        result3 = eval(formula)
        
    except Exception as e:
        print(code, ': {}'.format(type(e).__name__))

    # 汇总输出计算结果字典results
    results3[code] = pd.DataFrame(result3).reset_index()

for code, result3 in results3.items():
    result3.rename(columns={result3.columns[2]: code}, inplace=True)

for code, result3 in results3.items():
    df1 = pd.merge(df1, result3, on=['entity_id', 'year'], how='left')


print('===== p3指标计算完成 =====')


df1['MinProfitAndProfitAfterDedu']+df1['MinProfitAndProfitAfterDedu^1']+df1['MinProfitAndProfitAfterDedu^2']
===== 开始计算p3指标 =====
===== p3指标计算完成 =====


In [7]:
# 删除计算指标用的三表数据
drop_cols = df1.columns[7:61]
df1.drop(drop_cols, axis='columns', inplace=True)

In [8]:
# 调整列位置
df1.insert(1,'stock_code', df1.pop('stock_code'))
df1.insert(2,'stock_name', df1.pop('stock_name'))
df1.insert(3,'entity_name', df1.pop('entity_name'))

In [9]:
df1.head(2)

,entity_id,stock_code,stock_name,entity_name,year,uniform_credit,industry_whitewash,NOTES_AOO_AuditOpinion_idou,EN_ANNUAL_MarketCapitalization_idou,AverageRevenue,...,EBITDAToRevenue,MarginToMainIncome,CostAndExpenseToMainIncome,IntBearLiabilityIncreaseRate,IntBearLiabilityToNA,DistributableProfitsToInterest,EBITToTA,MinProfitAndProfitAfterDedu,Sum3YRDInvestmentToRevenue,SumMinProfitAndProfitAfterDedu
0,19718,000002.SZ,万科A,万科企业股份有限公司,2021,91440300192181490G,M009,NaN,NaN,NaN,...,0.144802,0.218245,0.890265,NaN,0.686944,NaN,NaN,3.792728e+10,NaN,NaN
1,19719,000004.SZ,国农科技,深圳国华网安科技股份有限公司,2021,91440300192441969E,M017,NaN,NaN,2.265434e+11,...,-1.710403,0.560689,0.744238,-0.999991,0.002552,-419713.347478,-0.00052,-5.206705e+08,NaN,NaN


In [10]:
save2csv(df1, '2_计算好指标的主体')